<img src="img/datenschule_logo_web.png" width="25%"  align="left">

<img src="img/Coat_of_arms_of_Berlin.svg" width="11%"  align="right">
# Aktivitäten und Partnerschaften an Berliner Schulen
In diesem Notebook dokumnetieren wir unser Vorgehen bei der Auswertung der Partnerschaften und Aktivitäten <br> der Berliner Schulen. Da wir die Daten auf der Website  laufend aktualisieren, können die Ergebnisse hier <br> von denen auf der Website abweichen. Der Berechnungsweg ist allerdings der Gleiche.
 <br> Mehr Informationen über das Schulsystem in Deutschland gibt es auf <a href='https://jedeschule.de'>jedeschule.de<a>

In [1]:
#Für die Berechnung nutzen wir hier die Pandas-Library
import pandas as pd

### 1. Auswertung der Schulaktivitäten in Berlin

In [45]:
df_berlin_ag = pd.read_csv('data/berlin/ag_berlin.csv', sep=',')

Zunächst werden die CSV-Datei zu den Berliner AGs importiert. Die Daten liegen in unserem Github Ordner.

In [46]:
df_berlin_ag.head(5)

,id,ag_name,ag_cat
0,BE-15586,Astronomie,Naturwissenschaft / Technik
1,BE-15586,Astrophysik,Naturwissenschaft / Technik
2,BE-15586,Literatur,Literatur / Medien
3,BE-15586,Bildende Kunst,Kunst / Kultur
4,BE-15590,Rudern,Sport


Zu Beginn einer Analyse ist es sinnvoll, einen Überblick über die Daten zu gewinnen.

In [47]:
df_berlin_ag.describe(include='all')

,id,ag_name,ag_cat
count,3341,3341,3341
unique,413,555,10
top,BE-16471,Fußball,Sport
freq,27,215,1124


Mit dem Befehl *'describe'* können erhalten wir eine Zusammenfassung der Daten. Wir sehen: 
 * *unique*: 413 der 800 allgemeinbildenden Schulen in Berlin kommunizieren ihre Projekte und Aktivitäten Projekte und Aktivitäten an die Senatsverwaltung.
 * *count*: Es werden über 3300 Aktivitäten an Berliner Schulen angeboten.

In [49]:
# Anzahl der Schulen in Berlin
schools_count = len(df_berlin_ag.id.unique())

# Anzahl der Angebotenen AGs
projects_count = len(df_berlin_ag)

# Durchschnittliche Anzahl an Projekten an Beriner Schulen
projects_average = projects_count / schools_count
projects_average

8.08958837772397

Um zu errechnen wie viele Projekte eine Schule in Berlin durchschnittlich anbietet, teilen wir die Anzahl der Projekte durch die Anzahl der Schulen. Da die CSV-Datei für jedes Angebot einen eigenen Zeileneintrag enthält, werden auch Schulen doppelt genannt. Mit der Funktion *'unique()'* filtern wir doppelte Schuleinträge aus. Dadurch erhalten wir die Anzahl der Schulen in Berlin, die ihre Aktivitäten kommuniziert haben. <br>
**Ergebnis:**
* Durchschnittlich bietet eine Schule in Berlin ihren Schüler*innen rund 8 Aktivitäten an.

In [51]:
#Wie viel Projekte aus unterschiedlichen Bereichen bieten die Schulen ihren Schüler*innen?
category_unique = df_berlin_ag.drop_duplicates(subset=['id','ag_cat'])

#Durchschnittliche Anzahl von Aktivitätskategorien pro Schule
category_avg = len(category_unique) / schools_count
print('In Berlin werden in durchschnittlich %s Kategorien Aktivitäten angeboten.' %(round(category_avg,2)))

In Berlin werden in durchschnittlich 4.1 Kategorien Aktivitäten angeboten.


In [52]:
len(category_unique)

1692

In [54]:
categories_pivot = pd.pivot_table(kategorie_unique,index=['ag_cat'],values='id',aggfunc='count')
df_categories = pd.DataFrame(categories_pivot)
df_categories['% Schulen mit Projekt-Kategorie'] = (df_categories.id / schools_count * 100).round(2)
df_categories.sort_values(by='id',ascending=False)

,id,% Schulen mit Projekt-Kategorie
ag_cat,,
Sport,355,85.96
Musik / Tanz,312,75.54
Kunst / Kultur,216,52.30
Naturwissenschaft / Technik,184,44.55
Handwerk,161,38.98
Literatur / Medien,158,38.26
Gesellschaft / Partizipation,105,25.42
Sprachen,105,25.42
Umwelt,77,18.64


Um zu errechen wie viele Schulen für eine jeweilige Kategorie Aktivitäten anbieten, nutzen wir die Pivot-Funktion. *(pivot_table())*. Mit der Funktion gruppieren wir die Aktivitäten nach ihren Kategorien. Teilt man diese Werte (hier unter Id) durch die Gesamtanzahl der Schulen, erhalten wir den prozentualen Anteil An Schulen die Aktivitäten in einer Kategorie anbieten.


### 2. Auswertung der Schulpartnerschaften in Berlin

In [55]:
df_berlin_partner = pd.read_csv('data/berlin/partner_berlin.csv', sep=';')

In [56]:
df_berlin_partner.head(5)

,Schule,ID,school_type,Partner_name,Type_grob,Type_mittel,Type_detail
0,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,AAP-Programm,Unbestimmt,Modell/Förderprogramm/Projekt,NaN
1,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Arbeitskreis Medizintechnik Berlin-Brandenburg,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Bildung/Uni/Hochsch...,NaN
2,OSZ Handel 1,BE-16262,Berufliches Gymnasium,Ausbildungszentrum ROC Nijmegen,Öffentliche Infrastruktur,Familien-/Bildungs-/Kulturzentrum,NaN
3,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Baumann Bildung & Qualifizierung (BBQ),Wirtschaftsakteur,Wirtschaftsakteur (Bildung),NaN
4,Louise-Schroeder-Schule (OSZ Bürowirtschaft un...,BE-15687,Berufliches Gymnasium,Bildung für Berlin und Brandenburg e.V.,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Verein


Auch hier ist es zunächst sinnvoll,einen Blick in die Daten zu werfen.

In [57]:
df_berlin_partner.describe(include='all')

,Schule,ID,school_type,Partner_name,Type_grob,Type_mittel,Type_detail
count,2976,2976,2976,2975,2976,2976,1598
unique,411,411,15,2123,7,23,23
top,Albrecht-von-Graefe-Schule (Integrierte Sekund...,BE-16610,Grundschule,Verein Berliner Kaufleute und Industrieller (V...,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Verein
freq,25,25,1127,29,1097,983,830


Über die *'describe()'* Funktion bekommen wir die Anzahl der Partnerschaften (2976) sowie die Anzahl der Schulen (411)

In [58]:
typ_partner = pd.DataFrame(df_berlin_partner.Type_grob.value_counts())
typ_partner = pd.DataFrame(pd.pivot_table(df_berlin_partner,index=['Type_grob'],values='Partner_name', aggfunc='count'))
typ_partner['Verteilung'] = (typ_partner / sum(typ_partner.Partner_name)).round(4)
typ_partner.sort_values(by='Partner_name', ascending=False)

,Partner_name,Verteilung
Type_grob,,
Gemeinnütziger Akteur,1097,0.3687
Öffentliche Infrastruktur,886,0.2978
Wirtschaftsakteur,551,0.1852
Partnerschule,246,0.0827
Unbestimmt,119,0.0400
Religiöse Einrichtung,42,0.0141
Verband / Kammer / Innung / Gewerkschaft,34,0.0114


In [59]:
schulkategorien = {'Grundschule' : [' Grundschule',
                                     'Grundschule'],
                    'Sekundarschule' : ['Integrierte Sekundarschule',
                                        ],
                    'Berufliche Schule' : ['Berufsfachschule',
                                           'Berufsoberschule',
                                           'Berufsschule',
                                           'Berufsschule mit sonderpäd. Aufgaben',
                                           'Fachoberschule',
                                           'Fachschule'],
                    'Förderschulen' : ['Förderschwerp."Geistige Entwicklung"',
                                       'Förderschwerp."Lernen"',
                                       'Übrige Förderschwerpunkte'
                                       ],
                    'Sonstige' :['Freie Waldorfschule',
                                 'Sonstige Ergänzungsschule',
                                 ],
                   
                   'Gymnasium' : ['Berufliches Gymnasium',
                                  'Gymnasien']
                   
                   }

Um die unterschiedlichen Schultypen zu vereinheitlichen, ordnen wir diese in 6 Kategorien ein: Grundschule, Sekundarschule, Berufliche Schule, Förderschulen, Gymnasium, Sonstige. Dazu erstellen wir das Dictionary: Schulkategorien.

In [60]:
def check_up_schultyp(schooltyp):
    for key, value in schulkategorien.items():
        for a in value:
            if a in str(schooltyp):
                return(key)

In [84]:
# Hier mappen wir die einzelnen Schultypen auf die 6 Schultypkategorien und säubern leere Zeilen aus.
df_berlin_partner['Schooltype_broad'] = df_Berlin.school_type.apply(check_up_schultyp)

In [85]:
df_berlin_partner.head(5)

,Schule,ID,school_type,Partner_name,Type_grob,Type_mittel,Type_detail,Schooltype_broad
0,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,AAP-Programm,Unbestimmt,Modell/Förderprogramm/Projekt,NaN,Gymnasium
1,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Arbeitskreis Medizintechnik Berlin-Brandenburg,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Bildung/Uni/Hochsch...,NaN,Gymnasium
2,OSZ Handel 1,BE-16262,Berufliches Gymnasium,Ausbildungszentrum ROC Nijmegen,Öffentliche Infrastruktur,Familien-/Bildungs-/Kulturzentrum,NaN,Gymnasium
3,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Baumann Bildung & Qualifizierung (BBQ),Wirtschaftsakteur,Wirtschaftsakteur (Bildung),NaN,Gymnasium
4,Louise-Schroeder-Schule (OSZ Bürowirtschaft un...,BE-15687,Berufliches Gymnasium,Bildung für Berlin und Brandenburg e.V.,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Verein,Gymnasium


In [86]:
#Durchscnittliche Anzahl an Partnerschaften pro Schule
berlin_unique = pd.DataFrame(df_berlin_partner.drop_duplicates())
len(berlin_unique) / len(berlin_unique.ID.unique())

7.238442822384428

In [87]:
partner_schulen = pd.DataFrame(pd.pivot_table(df_Berlin,index=['Schooltype_broad'],values='school_type',columns='Type_grob', aggfunc='count'))
partner_schulen = partner_schulen.fillna(0)

In [88]:
df_berlin_partner.head(2)

,Schule,ID,school_type,Partner_name,Type_grob,Type_mittel,Type_detail,Schooltype_broad
0,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,AAP-Programm,Unbestimmt,Modell/Förderprogramm/Projekt,NaN,Gymnasium
1,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Arbeitskreis Medizintechnik Berlin-Brandenburg,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Bildung/Uni/Hochsch...,NaN,Gymnasium


#### Wie viele Akteure für die jeweilige Kategorie kommen auf  eine Schule in Berlin?

In [89]:
dftyp = df_berlin_partner.drop_duplicates(subset=['ID','school_type'])
dftyp = pd.DataFrame(dftyp.Schooltype_broad.value_counts())

In [90]:
dftyp

,Schooltype_broad
Grundschule,164
Sekundarschule,99
Gymnasium,92
Berufliche Schule,36
Förderschulen,5
Sonstige,4


Hier errechnen wir die Anzahl der Schulen in Berlin für jeden Schultyp.

In [91]:
partner_schulen['Schulanzahl'] = dftyp

In [92]:
partner_schulen.head(2)

Type_grob,Gemeinnütziger Akteur,Partnerschule,Religiöse Einrichtung,Unbestimmt,Verband / Kammer / Innung / Gewerkschaft,Wirtschaftsakteur,Öffentliche Infrastruktur,Schulanzahl
Schooltype_broad,,,,,,,,
Berufliche Schule,45.0,9.0,4.0,20.0,10.0,59.0,43.0,36
Förderschulen,9.0,1.0,0.0,0.0,0.0,5.0,6.0,5


Anschließend fügen wir die Anzahl der Schulen pro Schultyp an das zuvorerstellte Dataframe mit der Anzahl der Partnerschaften pro Schultyp.

In [94]:
partner_schulen[['Gemeinnütziger Akteur','Partnerschule','Religiöse Einrichtung', 'Unbestimmt', 'Verband / Kammer / Innung / Gewerkschaft', 'Wirtschaftsakteur', 'Öffentliche Infrastruktur']].div(partner_schulen.Schulanzahl, axis=0)

Type_grob,Gemeinnütziger Akteur,Partnerschule,Religiöse Einrichtung,Unbestimmt,Verband / Kammer / Innung / Gewerkschaft,Wirtschaftsakteur,Öffentliche Infrastruktur
Schooltype_broad,,,,,,,
Berufliche Schule,1.250000,0.250000,0.111111,0.555556,0.277778,1.638889,1.194444
Förderschulen,1.800000,0.200000,0.000000,0.000000,0.000000,1.000000,1.200000
Grundschule,3.054878,0.164634,0.091463,0.231707,0.018293,0.810976,2.500000
Gymnasium,1.619565,1.826087,0.130435,0.293478,0.097826,1.358696,2.293478
Sekundarschule,3.646465,0.373737,0.070707,0.343434,0.111111,2.141414,2.040404
Sonstige,0.500000,0.500000,0.250000,0.000000,0.000000,1.750000,0.500000


Anschließend teilen wir die Anzahl der Partnerschaften pro Akteurskategorie durch die jeweilige Anzahl der Schulen pro Schultyp. Die obenstehende Tabelle zeigt also für jeden Schultyp wie viele Partnerschaften sie durchschnittlich pro Akteurskategorie eingeht.